In [ ]:
from datetime import datetime
import numpy as np
from cpuinfo import get_cpu_info
from Tools.PlotTools import VisualTool
from Tools.MapLoader import MapLoader
from Tools.Mask import layer_map

In [ ]:
vis=VisualTool(show=True, size=(5,5), save=False)
map=MapLoader().load("gangjin.crop1")

# Overview of Construction site map

In [ ]:
vis.showJetMap(map_data=map, title="Construction Site Map")

Installable map
<br><sub> A binary grid mask that defines physically feasible sensor deployment locations on the site. </sub>

In [ ]:
installable_layer = layer_map(map, keep_values=[2])
vis.showJetMap(map_data=installable_layer, title="Installable grids on map")

Uninstallable map
<br><sub>Regions excluded from sensor deployment due to safety, accessibility, or operational constraints <br>(e.g., equipment routes, waterlogged areas, vegetation, and offices).</sub>

In [ ]:
road_layer = layer_map(map, keep_values=[3])
vis.showJetMap(map_data=road_layer, title="Uninstallable(restricted) grids on map")

Jobsite area
<br><sub>The coverage target area for sensor deployment, defined as the union of installable and non-installable regions.</sub>

In [ ]:
jobsite_layer = layer_map(map, keep_values=[2,3])
vis.showJetMap(map_data=jobsite_layer, title="Jobsite grids on map")

In [ ]:
#Set sensor coverage radius
COVERAGE = 45

Outermost corner location candidate extraction
<br><sub> Detection of outermost feasible sensor placement candidates within installable regions, since environmental sensing modules must be deployed at the site boundaries to capture extreme conditions. </sub>

In [ ]:
from OuterDeployment.HarrisCorner import HarrisCorner

corner_layer = HarrisCorner(jobsite_layer)
corner_candidate = corner_layer.run(
    grid=jobsite_layer,                
    installable_layer=installable_layer, 
    blockSize=3,
    ksize=3,
    k=0.01,
    dilate_size=5,
    min_dist=5,
)

print("Outmost corner points:", len(corner_candidate))
vis.showJetMap_circle(map, corner_candidate, radius=COVERAGE)

Inner-Deployment Strategy: Genetic Algorithm
<br><sub>Genetic Algorithm is used to optimize the placement of inner sensors, minimizing the number of sensors while ensuring sufficient coverage of the jobsite area. </sub>

In [ ]:
from InnerDeployment.GeneticAlgorithm.main import SensorGA

optimized_result = SensorGA(
                installable_map=installable_layer,
                jobsite_map=jobsite_layer,
                coverage=COVERAGE,
                generations=100,
                corner_positions=corner_candidate,
                initial_size=100,
                selection_size=50,
                child_chromo_size=100,
                min_sensors=50,
                max_sensors=100,
            ).run(
                    selection_method="tournament",
                    mutation_rate=0.7,
                    early_stop=True,
                    early_stop_coverage=90.0,
                    early_stop_patience=5,
                    return_best_only=True
                )

In [ ]:
final_result = optimized_result + corner_candidate
print("Total number of Sensor :", len(final_result))
vis.showJetMap_circle(
    map_data=map, 
    sensor_positions=final_result,
    radius=COVERAGE
    )